# 1. TimeStamp, Caption, Label 불러오기

In [65]:
### read csv --------------
import numpy as np
import pandas as pd

textdata = pd.read_csv("./input_text/testdata.csv")
textdata.head(2)

,Start Time,End Time,Text,Lable
0,00:00:05:26,00:00:08:08,처음부터 혼자가 하려던 건 아니었다,neutral
1,00:00:24:24,00:00:27:22,연결이 되지 않아 사서함으로 연결되며,neutral


In [66]:
### 자막 --------------
text_list = []
for i in textdata["Text"] : text_list.append(i)
print(text_list[:3])

['처음부터 혼자가 하려던 건 아니었다', '연결이 되지 않아 사서함으로 연결되며', '삐소리 이후 통행료가 부과됩니다']


In [67]:
texts = []
for i in range (0, 6):
    texts.append(text_list[i])
print(texts)

['처음부터 혼자가 하려던 건 아니었다', '연결이 되지 않아 사서함으로 연결되며', '삐소리 이후 통행료가 부과됩니다', '얼마나 기다린 거지?', '그냥 갈까?', '결국 혼자다']


In [68]:
### 타임스탬프 --------------
textdataList = textdata.values.tolist()

print(textdataList[0:3])
start_times, end_times = [], []
for times in textdataList :
    start_times.append(times[0])
    end_times.append(times[1])

print(start_times[:3], end_times[:3])

[['00:00:05:26', '00:00:08:08', '처음부터 혼자가 하려던 건 아니었다', 'neutral'], ['00:00:24:24', '00:00:27:22', '연결이 되지 않아 사서함으로 연결되며', 'neutral'], ['00:00:28:07', '00:00:32:21', '삐소리 이후 통행료가 부과됩니다', 'neutral']]
['00:00:05:26', '00:00:24:24', '00:00:28:07'] ['00:00:08:08', '00:00:27:22', '00:00:32:21']


In [69]:
### 타임스탬프 -> 초 단위로 변환 --------------
def convert_to_seconds(time):
    h, m, s, ms = map(int, time.split(':'))
    return 3600 * h + 60 * m + s + ms / 100.0

startTimes = [convert_to_seconds(time) for time in start_times]
endTimes = [convert_to_seconds(time) for time in end_times]

print(startTimes[:5])
print(endTimes[:5])

[5.26, 24.24, 28.07, 33.26, 36.05]
[8.08, 27.22, 32.21, 36.05, 37.24]


In [70]:
start_times, end_times = [], []
# 1분 이상(60초) 타임스탬프 제외
for start in startTimes:
    if start < 60 :
        start_times.append(start)
    else :
        pass

for end in endTimes:
    if end < 60 :
        end_times.append(end)
    else :
        pass
print(start_times)
print(end_times)

[5.26, 24.24, 28.07, 33.26, 36.05, 51.13]
[8.08, 27.22, 32.21, 36.05, 37.24, 54.17]


# 2. 자막 이미지 변환

In [ ]:
### 자막 이미지 생성 ----------------------------
# height = 폰트 세로 크기로 리사이즈
# width = 비디오 가로 크기로 리사이즈

from PIL import Image, ImageFont, ImageDraw 

# Load the font
font = ImageFont.truetype("./NotoSansKR-Black.otf", 38)

for i, text in enumerate(texts):
  # Get the width and height of the text in pixels
  textwidth, textheight = ImageDraw.Draw(Image.new('RGBA', (500,500), (255,255,255,0))).textsize(text, font)

  # Create a blank image with the width of 1280 and height equal to the height of the text
  img = Image.new('RGBA', (1280, textheight), (255,255,255,0))

  # Create a draw object
  draw = ImageDraw.Draw(img)

  # Calculate the x and y coordinates to center the text in the image
  x = (1280 - textwidth) / 2
  y = (textheight - textheight) / 2

  # Draw the text on the image
  draw.text((x, y), text, font=font, fill=(255,255,255,255))

  # Save the image
  img.save(f"./imglist/{i}.png")

In [63]:
### 오름차순으로 리스트 담기
import os
from glob import glob

folder_path = './imglist'
image_extensions = 'png'

def is_image(filename):
    return any(filename.endswith(ext) for ext in image_extensions)

subtitle_images_list = []
subtitle_images_list = sorted([f for f in glob(os.path.join(folder_path, '*')) if is_image(f)])
subtitle_images_list = sorted(subtitle_images_list, key=lambda x: int(os.path.splitext(os.path.basename(x))[0]))

print(subtitle_images_list)

['./imglist/0.png', './imglist/1.png', './imglist/2.png', './imglist/3.png', './imglist/4.png', './imglist/5.png']


# 감정표현 효과 합성

In [58]:
### 오름차순으로 리스트 담기
import os
from glob import glob

folder_path = './input_img'
image_extensions = 'png'

def is_image(filename):
    return any(filename.endswith(ext) for ext in image_extensions)

subtitle_images_list = []
subtitle_images_list = sorted([f for f in glob(os.path.join(folder_path, '*')) if is_image(f)])
# subtitle_images_list = sorted(subtitle_images_list, key=lambda x: int(os.path.splitext(os.path.basename(x))[0]))

print(subtitle_images_list)

['./input_img/angry.png']


In [59]:
# 자막 이미지 리스트

subtitle_images = []
for subtitle_image_file in subtitle_images_list:
    subtitle_image = cv2.imread(subtitle_image_file)
    subtitle_images.append(subtitle_image)

subtitle_images = []
subtitle_heights = []
subtitle_widths = []
for subtitle_image_file in subtitle_images_list:
    subtitle_image = cv2.imread(subtitle_image_file)
    subtitle_images.append(subtitle_image)
    subtitle_height, subtitle_width, _ = subtitle_image.shape
    subtitle_heights.append(subtitle_height)
    subtitle_widths.append(subtitle_width)
print(len(subtitle_images))


1


In [61]:
import cv2
import numpy as np


### 타임스탬프에 따른 영상과 자막 이미지 합성
start_times = [5.26]
end_times = [8.08]
input_video_path = "./input_video/test_1m.mp4"
video = cv2.VideoCapture(input_video_path)

# Get the dimensions of the video
video_width = int(video.get(3))
video_height = int(video.get(4))

output_file = "./output_video/angry.mp4"

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_file, fourcc, 30.0, (video_width, video_height))

frame_counter = 0
i = 0
while True:
    ret, frame = video.read()
    if not ret:
        break
    
    if i < len(start_times) and frame_counter / 20.0 >= start_times[i]:
        subtitle_image = subtitle_images[i]
        subtitle_height, subtitle_width, _ = subtitle_image.shape
        x_offset = (video_width - subtitle_width) // 2
        y_offset = video_height - subtitle_height
        roi = frame[y_offset:y_offset+subtitle_height, x_offset:x_offset+subtitle_width]
        img2gray = cv2.cvtColor(subtitle_image, cv2.COLOR_BGR2RGB)
        img2gray = cv2.cvtColor(img2gray, cv2.COLOR_RGB2GRAY)
        ret, mask = cv2.threshold(img2gray, 10, 255, cv2.THRESH_BINARY)
        mask_inv = cv2.bitwise_not(mask)
        img1_bg = cv2.bitwise_and(roi, roi, mask=mask_inv)
        img2_fg = cv2.bitwise_and(subtitle_image, subtitle_image, mask=mask)
        dst = cv2.add(img1_bg, img2_fg)
        frame[y_offset:y_offset+subtitle_height, x_offset:x_offset+subtitle_width] = dst
    
    out.write(frame)
    frame_counter += 1
    
    if i < len(end_times) and frame_counter / 20.0 >= end_times[i]:
        i += 1

out.release()


# 동영상과 자막 이미지 합성

In [71]:
### 정해진 TimeStamp에 따라 자막을 합성하는 함수 ----------------

import cv2
import numpy as np
from PIL import Image, ImageDraw, ImageFont

def add_subtitles_within_time_range(text, input_video_path, start_time, end_time):
    # 비디오 로드
    video = cv2.VideoCapture(input_video_path)

    # Get the dimensions of the video
    frame_width = int(video.get(3))
    frame_height = int(video.get(4))

    # Define the codec and create a VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*"mp4v") 
    out = cv2.VideoWriter("./output_video/output.mp4", fourcc, 30.0, (frame_width, frame_height))

    # Define the text font and color
    font = ImageFont.truetype(".//NotoSansKR-Black.otf", 36)
    color = (255, 255, 255)

    # Read and process each frame of the video
    start_frame = int(start_time * 30)
    end_frame = int(end_time * 30)
    current_frame = 0
    while True:
        ret, frame = video.read()
        if not ret:
            break
        if current_frame >= start_frame and current_frame <= end_frame:
            # Convert the frame to a PIL image
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame = Image.fromarray(frame)
            
            # Create a draw object and add text to the image
            draw = ImageDraw.Draw(frame)
            text = text
            textwidth, textheight = font.getsize(text)
            text_position = (frame_width / 2 - textwidth / 2, frame_height - textheight - 50)
            draw.text(text_position, text, font=font, fill=color)   
            
            # Convert the PIL image back to a OpenCV frame
            frame = np.array(frame)
            frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
            
        # Write the frame to the output video
        out.write(frame)
        current_frame += 1

    # Release the video and VideoWriter objects
    out.release()
    video.release()


In [72]:
import shutil # copy 생성 라이브러리
from tqdm import tqdm
import time

input_video_path = "./output_video/angry.mp4"
cnt = 0

for i in tqdm(range(len(texts)), desc=f'{len(texts)}개의 자막과 동영상을 합성 중 입니다.'):
    cnt += 1
    text = texts[i]
    start_time = start_times[i]
    end_time = end_times[i]
    if cnt == 1 :
        add_subtitles_within_time_range(text, input_video_path, start_time, end_time)
        input_file = "./output_video/output.mp4"
        output_file = "./output_video/copy.mp4"
        shutil.copy(input_file, output_file)

    else :
        input_video_path = "./output_video/copy.mp4"
        add_subtitles_within_time_range(text, input_video_path, start_time, end_time)
        input_file = "./output_video/output.mp4"
        output_file = "./output_video/copy.mp4"
        shutil.copy(input_file, output_file)


6개의 자막과 동영상을 합성 중 입니다.:   0%|          | 0/6 [00:00<?, ?it/s]/var/folders/jk/6v6fgrlx26s0xvwkcpch09xw0000gn/T/ipykernel_72435/3952835329.py:39: DeprecationWarning: getsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use getbbox or getlength instead.
  textwidth, textheight = font.getsize(text)
6개의 자막과 동영상을 합성 중 입니다.: 100%|██████████| 6/6 [01:19<00:00, 13.32s/it]


# 마지막 음성 재합성